In [1]:
import polars as pl

In [2]:
# stop_t = pl.scan_csv('IDFM-gtfs/stop_times.txt', separator=',')
# stops = pl.scan_csv('IDFM-gtfs/stops.txt', separator=',')
# trips = pl.scan_csv('IDFM-gtfs/trips.txt', separator=',')
# calendar = pl.scan_csv('IDFM-gtfs/calendar.txt', separator=',')
# calendar_dates = pl.scan_csv('IDFM-gtfs/calendar_dates.txt', separator=',')

In [77]:
# Parameters

# Columns
col_stop_t = ['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence', 'stop_sequence',	'pickup_type', 'drop_off_type', 'timepoint']
col_stops = ['stop_id', 'stop_name']
col_trips = ['route_id', 'service_id', 'trip_id', 'direction_id']
col_calendar = ['service_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'start_date', 'end_date']
col_calendar_dates = ['service_id', 'date', 'exception_type']

# RER B specific
line_b = 'C01743' # RER B's ID_Line

# EDA

## stop_times.txt

In [37]:
stop_t = pl.read_csv('IDFM-gtfs/stop_times.txt', separator=',')

In [40]:
stop_t.describe()

describe,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,local_zone_id,stop_headsign,timepoint
str,str,str,str,str,f64,f64,f64,str,str,f64
"""count""","""7102994""","""7102994""","""7102994""","""7102994""",7.102994e6,7.102994e6,7.102994e6,"""7102994""","""7102994""",7.102994e6
"""null_count""","""0""","""0""","""0""","""0""",0.0,0.0,0.0,"""7102994""","""7102994""",0.0
"""mean""",null,null,null,null,13.022972,0.04569,0.045787,null,null,1.0
"""std""",null,null,null,null,9.408493,0.208812,0.209023,null,null,0.0
"""min""","""IDFM:ADP:5340-…","""00:00:00""","""00:00:00""","""IDFM:10001""",0.0,0.0,0.0,null,null,1.0
"""25%""",null,null,null,null,5.0,0.0,0.0,null,null,1.0
"""50%""",null,null,null,null,12.0,0.0,0.0,null,null,1.0
"""75%""",null,null,null,null,19.0,0.0,0.0,null,null,1.0
"""max""","""IDFM:stif:loca…","""39:40:00""","""39:40:00""","""IDFM:monomodal…",84.0,1.0,1.0,null,null,1.0


In [52]:
stop_t.select('trip_id').n_unique()

324058

Trip_id is not the primar key  which is obvious because there are several stops for a single trip.
Let's try trip_id x stop_id

In [54]:
stop_t.select(['trip_id', 'stop_id']).n_unique()

7094671

Let's crunch the duplicated rows

In [62]:
stop_t.filter(pl.struct(['trip_id', 'stop_id']).is_duplicated()).sort(by=['trip_id', 'stop_id', 'arrival_time'])

trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type,local_zone_id,stop_headsign,timepoint
str,str,str,str,i64,i64,i64,str,str,i64
"""IDFM:ADP:5340-…","""04:00:00""","""04:00:00""","""IDFM:10228""",0,0,1,null,null,1
"""IDFM:ADP:5340-…","""04:15:00""","""04:15:00""","""IDFM:10228""",9,1,0,null,null,1
"""IDFM:ADP:5340-…","""04:07:00""","""04:07:00""","""IDFM:10228""",0,0,1,null,null,1
"""IDFM:ADP:5340-…","""04:22:00""","""04:22:00""","""IDFM:10228""",9,1,0,null,null,1
"""IDFM:ADP:5340-…","""04:14:00""","""04:14:00""","""IDFM:10228""",0,0,1,null,null,1
"""IDFM:ADP:5340-…","""04:29:00""","""04:29:00""","""IDFM:10228""",9,1,0,null,null,1
"""IDFM:ADP:5340-…","""04:21:00""","""04:21:00""","""IDFM:10228""",0,0,1,null,null,1
"""IDFM:ADP:5340-…","""04:36:00""","""04:36:00""","""IDFM:10228""",9,1,0,null,null,1
"""IDFM:ADP:5340-…","""04:28:00""","""04:28:00""","""IDFM:10228""",0,0,1,null,null,1


What are the meaning of the changing columns ? stop_sequence : numéro d'ordre d'arret de la course, pickup_type	: possibilité ou non de monter, drop_off_type : possibilité ou non de descendre

What are even trip_ids ? Let's see inside the trip.txt file

# trips.txt

In [66]:
trips = pl.read_csv('IDFM-gtfs/trips.txt', separator=',', n_rows=1000)

In [75]:
trips.glimpse()

Rows: 324058
Columns: 10
$ route_id              <str> 'IDFM:C01740', 'IDFM:C01740', 'IDFM:C01740', 'IDFM:C01740', 'IDFM:C01740', 'IDFM:C01740', 'IDFM:C01740', 'IDFM:C01740', 'IDFM:C01740', 'IDFM:C01740'
$ service_id            <str> 'IDFM:TN:003704bf-64ed-39e3-b539-089843a0f572', 'IDFM:TN:003704bf-64ed-39e3-b539-089843a0f572', 'IDFM:TN:003704bf-64ed-39e3-b539-089843a0f572', 'IDFM:TN:003704bf-64ed-39e3-b539-089843a0f572', 'IDFM:TN:003704bf-64ed-39e3-b539-089843a0f572', 'IDFM:TN:003704bf-64ed-39e3-b539-089843a0f572', 'IDFM:TN:003704bf-64ed-39e3-b539-089843a0f572', 'IDFM:TN:003704bf-64ed-39e3-b539-089843a0f572', 'IDFM:TN:003704bf-64ed-39e3-b539-089843a0f572', 'IDFM:TN:003704bf-64ed-39e3-b539-089843a0f572'
$ trip_id               <str> 'IDFM:TN:SNCF:9d68d0f4-c7cf-40de-8095-641cd642d1b0', 'IDFM:TN:SNCF:4521fae6-46e9-4298-955a-57cbc5c9ee98', 'IDFM:TN:SNCF:a295e58b-19d5-4269-9706-7d96d12d74e5', 'IDFM:TN:SNCF:902bc3da-0114-4115-aea2-596039adb6ec', 'IDFM:TN:SNCF:7eb4b422-0033-440e-92b1-1c5f674

In [71]:
trips = pl.read_csv('IDFM-gtfs/trips.txt', separator=',', ignore_errors=True) # last row is empty 

In [74]:
trips.select(['trip_id']).n_unique()

324058

trip_id is the primary key

In [76]:
trips.select(['service_id']).n_unique()

722

In [ ]:
Pour une route(ligne) donnée, il a plusieurs trips (parcours) possibles. Vérifions cela pour un cas particulier, la ligne B du RER.

In [94]:
# ligne_b is a substring of route_id
# How many different trips for line B ?
trips.filter(pl.col('route_id').str.contains(line_b)).select(['trip_id']).n_unique()

3209

In [98]:
# That's a lot. Maybe same itinary on different time are two different trips ? Let's take one trip_id and see if there are multiple arrival_times possible
trips.filter(pl.col('route_id').str.contains(line_b)).select(['trip_id']).sort(by='trip_id').glimpse()

Rows: 3209
Columns: 1
$ trip_id <str> 'IDFM:TN:SNCF:001cb759-ef2a-4264-adca-051689614aa3', 'IDFM:TN:SNCF:00202fe9-825c-40a2-988f-a59ff1df63c1', 'IDFM:TN:SNCF:0028f19e-8758-4380-8290-d5eb44206db7', 'IDFM:TN:SNCF:0053481c-9b20-4459-9c49-2bd78486fc8d', 'IDFM:TN:SNCF:00718c5a-1e22-4673-83d2-26e7ce7444ff', 'IDFM:TN:SNCF:0078afab-35ce-47ef-9cd6-a48b6453eccc', 'IDFM:TN:SNCF:0087eddb-49a7-466b-8da8-116ce66317c1', 'IDFM:TN:SNCF:008d9898-3e75-463b-a48f-abb47083eb2f', 'IDFM:TN:SNCF:00c982e8-8ac8-4bf6-a280-6d0ddee14f7d', 'IDFM:TN:SNCF:00cfcdd1-e95a-4e15-9c91-fec3dafaf622'



In [100]:
# for trip_id = "IDFM:TN:SNCF:001cb759-ef2a-4264-adca-051689614aa3" , let's see in stop_times.txt if there are multiple arrival_time
stop_t.filter(pl.col('trip_id') == "IDFM:TN:SNCF:001cb759-ef2a-4264-adca-051689614aa3").select(['arrival_time']).sort(by='arrival_time').glimpse()

Rows: 33
Columns: 1
$ arrival_time <str> '17:37:02', '17:38:45', '17:40:52', '17:43:21', '17:45:12', '17:47:24', '17:50:54', '17:52:29', '17:54:30', '17:56:21'



In [35]:
# Actually, this might be for different date. We really need to create the whole consolidated dataframe. 

trip_id
str
"""IDFM:KABS:1159…"
"""IDFM:KSOE:1071…"
"""IDFM:RATP:1076…"
"""IDFM:TICE:1137…"
"""IDFM:RATP:1075…"
"""IDFM:RATP:1074…"
"""IDFM:RATP:1164…"
"""IDFM:RATP:1077…"
"""IDFM:TN:SNCF:c…"


NameError: name 'dsds' is not defined

: 